In [1]:
!pip install pympler

In [2]:
import datetime
hora_inicio_codigo = datetime.datetime.now()
print(hora_inicio_codigo)

2024-11-22 09:59:59.161064


In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

import csv
import math
import os
import chardet
import shap
from tqdm import tqdm

from imblearn.over_sampling import SMOTE
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import KFold, GridSearchCV, StratifiedKFold, train_test_split, cross_val_predict
from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error

from pympler import asizeof
import joblib

In [ ]:
data_file = './csv_final/final_data_numeric.csv'

with open(data_file, 'rb') as file:
    resultado = chardet.detect(file.read(10000))

# Ver la codificación detectada
print(f"Codificación detectada: {resultado['encoding']}")

# Leer el archivo CSV con la codificación detectada
data = pd.read_csv(data_file, sep=';', encoding=resultado['encoding'])

y_file = './csv_final/result.csv'

# Ver la codificación detectada
print(f"Codificación detectada: {resultado['encoding']}")

# Leer el archivo CSV con la codificación detectada
y = pd.read_csv(y_file, sep=';', encoding=resultado['encoding'])

Codificación detectada: utf-8
Codificación detectada: utf-8


# Separamos en X_Train, X_Test, y_train, y_test

In [5]:
# Definición de las columnas para cada subset (ajustado según los nombres exactos)
demographic_columns = [
    'por_favor_indique_el_genero_con_el_que_se_identifica:',  # Gender
    'indique_su_estado_civil:',  # Marital Status
    '¿tiene_la_nacionalidad_europea?',  # Nationality
    '¿cual_es_el_estado_actual_de_su_documentacion?',  # Current Documentation Status
    '¿cual_es_el_maximo_nivel_educativo_que_ha_terminado?',  # Education Level
    'por_favor,_indique_su_estructura_de_convivencia_habitual_durante_los_12_meses_anteriores_al_confinamiento:',  # Family Structure before COVID
    'por_favor,_indique_su_estructura_de_convivencia_habitual_durante_la_reclusion:',  # Family Structure during COVID
    'por_favor_indique_el_numero_de_menores_a_su_cargo_durante_la_cuarentena:',  # Number of Minors in Care During COVID
    'por_favor_indique_el_numero_de_personas_dependientes_mayores_de_18_años_a_su_cargo_(incluyendo_personas_mayores,_personas_con_diversidad_funcional)_durante_la_cuarentena:',  # Number of Dependents
    'por_favor,_indique_con_cuanta_gente_tenia_contacto_habitualmente_en_un_dia_normal_antes__de_la_cuarentena_(incluyendo_el_hogar,_lo_laboral,_lo_social...)._nos_referimos_a_contactos_individuales_cara_a_cara,_es_decir,_con_las_que_charla_o_trata_diferentes_asuntos_en_persona.',  # Social Contacts Before COVID
    'por_favor,_indique_con_cuanta_gente_tiene/tuvo_contacto_habitualmente_en_un_dia_normal_durante_la_cuarentena__(incluyendo_el_hogar,_lo_laboral,_lo_social...)._nos_referimos_a_contactos_individuales_cara_a_cara,_es_decir,_con_las_que_charla_o_trata_diferentes_asuntos_en_persona.'  # Social Contacts During COVID
    ]

living_columns = [
    'por_favor,_indique_el_espacio_en_el_que_vive/vivia_durante_el_confinamiento:',  # Type of Living Space During Confinement
    'por_favor_indique_si_es_dueño_de_su_vivienda_o_vive_alquilado/a:',  # Property Ownership
    '¿cuantos_metros_cuadrados_utiles_(los_que_puede_pisar)_tiene/tenia_su_espacio_de_residencia_durante_la_cuarentena?',  # Square Meters of Living Space
    'diria_que_su_espacio_de_residencia_durante_la_cuarentena_tiene:',  # Light During Quarantine
    'diria_que_su_espacio_de_residencia_durante_la_cuarentena_tiene:.1',  # Ventilation During Quarantine
    'teniendo_en_mente_su_lugar_de_residencia_durante_el_confinamiento,_por_favor,_indique._numero_de_habitaciones_sin_contar_baño_y_cocina:',  # Number of Rooms
    'teniendo_en_mente_su_lugar_de_residencia_durante_el_confinamiento,_por_favor,_indique._numero_de_personas_con_las_que_cohabitaba/cohabita:',  # Number of Cohabitants
    'teniendo_en_mente_su_lugar_de_residencia_durante_el_confinamiento,_por_favor,_indique._numero_de_personas_en_su_mismo_dormitorio_excluyendose_a_usted_mismo/a:',  # Number of Cohabitants in the Same Room
    '¿cuantas_veces_a_la_semana_ha_salido_de_casa_durante_la_cuarentena?',  # Outings During Quarantine
    'la_idea_de_que_deberia_ser_castigado/a_por_sus_pecados',  # Assessment of Self Punishment
]

economic_columns = [
    'situacion_laboral_antes_(en_los_12_meses_anteriores_al_inicio_de_la_cuarentena_o_la_mayor_parte_de_ese_tiempo)',  # Employment Status Before Quarantine
    '¿que_duracion_tenia_su_jornada_laboral_antes_(en_los_12_meses_anteriores_al_inicio_de_la_cuarentena_o_la_mayor_parte_de_ese_tiempo):',  # Working Hours Before Quarantine
    '¿cual_ha_sido_su_situacion_laboral_durante_la_cuarentena?',  # Employment Status During Quarantine
    '¿cual_considera_que_podria_ser_su_situacion_laboral_o_academica_despu√âs_de_la_crisis_provocada_por_el_coronavirus_(en_los_12_meses_posteriores_al_fin_de_la_cuarentena_o_la_mayor_parte_de_ese_tiempo):?',  # Employment Status After Quarantine
    'en_los_ultimos_12_meses_antes_del_confinamiento,_¿cual_fue_la_principal_ocupacion_de_la_persona_que_aporta_un_mayor_importe_economico_al_hogar?',  # Occupation of the Person with the Highest Economic Contribution
    '¿cual_era_el_nivel_aproximado_de_ingresos_mensuales_netos_regulares_en_su_hogar_(unidad_en_la_que_se_comparten_gastos:_individuo,_pareja,_familia...)_antes_de_la_cuarentena_(en_los_12_meses_anteriores_al_inicio_de_la_cuarentena_o_la_mayor_parte_de_ese_tiempo):',  # Net Monthly Income Before COVID
    '¿cual_era_el_nivel_aproximado_de_ingresos_mensuales_netos_regulares_en_su_hogar_(unidad_en_la_que_se_comparten_gastos:_individuo,_pareja,_familia...)_durante_la_cuarentena:',  # Net Monthly Income During COVID
    'durante_el_confinamiento_en_su_hogar:',  # Household Conditions During Confinement
    'supongamos_que_usted_(y_su_conyuge_o_pareja)_convierte_en_dinero_todos_sus_fondos_en_cuentas_corrientes_y/o_de_ahorros,_inversiones_en_bolsa,_bonos/letras,_bienes_inmuebles,_y_venden_su_casa,_sus_vehiculos_y_todas_sus_cosas_de_valor._a_continuacion,_supongamos_que_el_dinero_de_todas_estas_operaciones_lo_utiliza_para_pagar_su_hipoteca_y_otros_creditos,_prestamos,_deudas_y_tarjetas_de_credito._¿le_quedaria_dinero_despues_de_pagar_todas_sus_deudas,_o_todavia_deberia_dinero?_(basta_con_un_estimacion_aproximada).',  # Financial Assessment After Liquidation
]

health_columns = [
    'condicion_de_salud',  # Health Condition and Specific Needs
    'recibe_tratamiento_o_medicacion_psiquiatrica_actualmente:',  # Psychiatric Treatment
    'recibe_tratamiento_psicologico_o_sigue_una_terapia_actualmente:',  # Psychological Treatment
    'por_favor_indique_si_necesita_ayuda_para_las_tareas_cotidianas_y_de_autocuidado_como_hacer_la_compra,_realizar_las_tareas_del_hogar,_lavarse_y_peinarse,_cocinar,_gestionar_el_dinero,_etc.',  # Help for Daily Tasks
    'por_favor_indique,_si_lo_hay,_el_grado_de_discapacidad_que_indique_su_certificado_%_reconocido_(0_si_no_tiene):',  # Chronic Illness
    'por_favor,_indique_si_en_el_pasado_usted_ha_tenido_algun_intento_de_suicidio:',
    'por_favor_indique_si_le_han_diagnosticado_de_infeccion_por_coronavirus',
    'en_el_caso_de_tener_o_haber_tenido_un_diagnostico_de_coronavirus,_¿como_valoraria_la_severidad_de_la_enfermedad?',
    'en_este_caso,_¿permanecio_aislado/a_dentro_del_domicilio_(sin_salir_de_una_habitacion_y_sin_compañia_durante_la_duracion_de_los_sintomas_y_15_dias_mas)?',
    'alguna_persona_de_su_familia_diagnosticada_de_infeccion_por_coronavirus,_¿ha_convivido_en_su_hogar_durante_el_proceso_de_su_enfermedad?',
    'en_caso_afirmativo,_valore,_por_favor,_la_severidad_de_la_enfermedad_(tenga_en_cuenta_el_de_mayor_gravedad_en_caso_de_ser_varios_casos)',
    'familiar_covid'
]

In [6]:
# Crear subconjuntos de datos
demographic_df = data[demographic_columns]
living_df = data[living_columns]
economic_df = data[economic_columns]
health_df = data[health_columns]

# Guardar los subconjuntos en archivos CSV
demographic_df.to_csv('./sets/demographic_data.csv', sep=';', index=False)
#display(demographic_df)
living_df.to_csv('./sets/living_data.csv', sep=';', index=False)
#display(living_df)
economic_df.to_csv('./sets/economic_data.csv', sep=';', index=False)
#display(economic_df)
health_df.to_csv('./sets/health_data.csv', sep=';', index=False)
#display(health_df)

datasets = [demographic_df, living_df, economic_df, health_df]

print("Subconjuntos de datos guardados exitosamente.")

Subconjuntos de datos guardados exitosamente.


In [7]:
# Directorios
input_dir = './sets'
output_dir = './entrenamiento'

# Asegurarse de que el directorio de salida existe
os.makedirs(output_dir, exist_ok=True)

# Leer los archivos en el directorio './sets'
sub_sets = os.listdir(input_dir)

sub_sets = [x for x in sub_sets if x.endswith('.csv')]

print("Archivos en './sets':", sub_sets)

Archivos en './sets': ['demographic_data.csv', 'economic_data.csv', 'health_data.csv', 'living_data.csv']


In [8]:
datasets = []

# Cargar todos los datasets en la lista datasets
for sub_set in sub_sets:
    data_file = f'{input_dir}/{sub_set}'
    
    with open(data_file, 'rb') as file:
        resultado = chardet.detect(file.read(10000))

    # Ver la codificación detectada
    print(f"Codificación detectada para {sub_set}: {resultado['encoding']}")
    
    # Leer el archivo CSV con la codificación detectada
    data = pd.read_csv(data_file, sep=';', encoding=resultado['encoding'])
    datasets.append((sub_set, data))  # Guarda el nombre del archivo y el DataFrame

Codificación detectada para demographic_data.csv: EUC-JP
Codificación detectada para economic_data.csv: utf-8
Codificación detectada para health_data.csv: EUC-JP
Codificación detectada para living_data.csv: EUC-JP


In [ ]:
# Leer el archivo 'result.csv' para obtener las columnas de interés
y_file = './csv_final/result.csv'

with open(y_file, 'rb') as file:
    resultado = chardet.detect(file.read(10000))

# Ver la codificación detectada
print(f"Codificación detectada para 'result.csv': {resultado['encoding']}")

# Leer el archivo CSV con la codificación detectada
y = pd.read_csv(y_file, sep=';', encoding=resultado['encoding'])

# Almacenar los nombres de las columnas en un array
columnas = y.columns.to_list()
print("Nombres de las columnas en 'result.csv':", columnas)

Codificación detectada para 'result.csv': utf-8
Nombres de las columnas en 'result.csv': ['Hostilidad', 'Somatización', 'Depresión', 'Obsesión-compulsión', 'Ansiedad', 'Sensibilidad interpersonal', 'Agorafobia', 'Ideación paranoide', 'Psicoticismo']


In [10]:
# Crear nuevos datasets combinando cada sub_set con cada columna de y
for sub_set_name, dataset in datasets:
    for columna in columnas:
        # Crear un nuevo DataFrame con la columna de y y los datos del dataset original
        new_dataset = dataset.copy()  # Hacer una copia del dataset original
        new_dataset[columna] = y[columna]  # Añadir la columna de 'y' al nuevo dataset
        
        # Guardar el nuevo DataFrame en un nuevo archivo CSV
        output_file = f'{output_dir}/{sub_set_name[:-4]}_with_{columna}.csv'
        new_dataset.to_csv(output_file, sep=';', index=False, encoding='utf-8')
        
        print(f"Archivo guardado: {output_file}")

Archivo guardado: ./entrenamiento/demographic_data_with_Hostilidad.csv
Archivo guardado: ./entrenamiento/demographic_data_with_Somatización.csv
Archivo guardado: ./entrenamiento/demographic_data_with_Depresión.csv
Archivo guardado: ./entrenamiento/demographic_data_with_Obsesión-compulsión.csv
Archivo guardado: ./entrenamiento/demographic_data_with_Ansiedad.csv
Archivo guardado: ./entrenamiento/demographic_data_with_Sensibilidad interpersonal.csv
Archivo guardado: ./entrenamiento/demographic_data_with_Agorafobia.csv
Archivo guardado: ./entrenamiento/demographic_data_with_Ideación paranoide.csv
Archivo guardado: ./entrenamiento/demographic_data_with_Psicoticismo.csv
Archivo guardado: ./entrenamiento/economic_data_with_Hostilidad.csv
Archivo guardado: ./entrenamiento/economic_data_with_Somatización.csv
Archivo guardado: ./entrenamiento/economic_data_with_Depresión.csv
Archivo guardado: ./entrenamiento/economic_data_with_Obsesión-compulsión.csv
Archivo guardado: ./entrenamiento/economic_da

In [11]:
# Directorio donde están los datasets
data_dir = './entrenamiento'
datasets = os.listdir(data_dir)
balanced_dir = './entrenamiento_balanceado'

# Crear el directorio si no existe
os.makedirs(balanced_dir, exist_ok=True)

# Filtrar archivos que no son CSV
datasets = [d for d in datasets if d.endswith('.csv')]

print(f'Datasets encontrados: {datasets}')

Datasets encontrados: ['demographic_data_with_Agorafobia.csv', 'demographic_data_with_Ansiedad.csv', 'demographic_data_with_Depresión.csv', 'demographic_data_with_Hostilidad.csv', 'demographic_data_with_Ideación paranoide.csv', 'demographic_data_with_Obsesión-compulsión.csv', 'demographic_data_with_Psicoticismo.csv', 'demographic_data_with_Sensibilidad interpersonal.csv', 'demographic_data_with_Somatización.csv', 'economic_data_with_Agorafobia.csv', 'economic_data_with_Ansiedad.csv', 'economic_data_with_Depresión.csv', 'economic_data_with_Hostilidad.csv', 'economic_data_with_Ideación paranoide.csv', 'economic_data_with_Obsesión-compulsión.csv', 'economic_data_with_Psicoticismo.csv', 'economic_data_with_Sensibilidad interpersonal.csv', 'economic_data_with_Somatización.csv', 'health_data_with_Agorafobia.csv', 'health_data_with_Ansiedad.csv', 'health_data_with_Depresión.csv', 'health_data_with_Hostilidad.csv', 'health_data_with_Ideación paranoide.csv', 'health_data_with_Obsesión-compulsió

In [12]:
for data_file in datasets:
    # Cargar el dataset
    data_path = os.path.join(data_dir, data_file)
    data = pd.read_csv(data_path, sep=';')
    
    # Separar características y etiquetas
    X = data.iloc[:, :-1]  # Todas las columnas menos la última
    y = data.iloc[:, -1]   # Última columna

    # Aplicar SMOTE para aumentar la clase minoritaria
    smote = SMOTE(sampling_strategy='not minority', random_state=42)
    X_resampled, y_resampled = smote.fit_resample(X, y)
    
    # Determinar el tamaño objetivo de la clase mayoritaria (igual al tamaño de la clase minoritaria original)
    clase_min = y.value_counts().min()
    
    # Crear un DataFrame de las muestras resampleadas para manipulación
    resampled_data = pd.DataFrame(X_resampled, columns=X.columns)
    resampled_data[str(data_file).split('_')[-1][:-4]] = y_resampled  # Suponiendo que str(data_file).split('_')[-1][:-4] es el nombre de la columna de etiquetas
    
    # Filtrar la clase mayoritaria y ordenar por la distancia a la media (u otra métrica)
    clase_mayoritaria = resampled_data[resampled_data[str(data_file).split('_')[-1][:-4]] == y.mode()[0]]
    clase_mayoritaria_mean = clase_mayoritaria.mean(axis=0)  # Calculamos la media de la clase mayoritaria
    
    # Ordenar la clase mayoritaria por proximidad a la media y seleccionar las primeras `clase_min` muestras
    clase_mayoritaria['distancia'] = clase_mayoritaria.drop(columns=str(data_file).split('_')[-1][:-4]).apply(
        lambda row: ((row - clase_mayoritaria_mean[:-1]) ** 2).sum(), axis=1)
    clase_mayoritaria_sorted = clase_mayoritaria.sort_values(by='distancia').head(clase_min)
    
    # Filtrar la clase minoritaria
    clase_minoritaria = resampled_data[resampled_data[str(data_file).split('_')[-1][:-4]] != y.mode()[0]]
    
    # Combinar clase mayoritaria balanceada y clase minoritaria
    balanced_data = pd.concat([clase_mayoritaria_sorted, clase_minoritaria]).drop(columns='distancia')
    
    # Guardar el dataset balanceado
    balanced_data_path = os.path.join(balanced_dir, f'balanceado_{data_file}')
    balanced_data.to_csv(balanced_data_path, sep=';', index=False)
    
    print(f'SMOTE y downsampling no aleatorio aplicados. Dataset guardado como: {balanced_data_path}')

C:\Users\Ceiec09\AppData\Local\Temp\ipykernel_7884\1170576296.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clase_mayoritaria['distancia'] = clase_mayoritaria.drop(columns=str(data_file).split('_')[-1][:-4]).apply(
C:\Users\Ceiec09\AppData\Local\Temp\ipykernel_7884\1170576296.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clase_mayoritaria['distancia'] = clase_mayoritaria.drop(columns=str(data_file).split('_')[-1][:-4]).apply(


SMOTE y downsampling no aleatorio aplicados. Dataset guardado como: ./entrenamiento_balanceado\balanceado_demographic_data_with_Agorafobia.csv
SMOTE y downsampling no aleatorio aplicados. Dataset guardado como: ./entrenamiento_balanceado\balanceado_demographic_data_with_Ansiedad.csv


C:\Users\Ceiec09\AppData\Local\Temp\ipykernel_7884\1170576296.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clase_mayoritaria['distancia'] = clase_mayoritaria.drop(columns=str(data_file).split('_')[-1][:-4]).apply(


SMOTE y downsampling no aleatorio aplicados. Dataset guardado como: ./entrenamiento_balanceado\balanceado_demographic_data_with_Depresión.csv


C:\Users\Ceiec09\AppData\Local\Temp\ipykernel_7884\1170576296.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clase_mayoritaria['distancia'] = clase_mayoritaria.drop(columns=str(data_file).split('_')[-1][:-4]).apply(
C:\Users\Ceiec09\AppData\Local\Temp\ipykernel_7884\1170576296.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clase_mayoritaria['distancia'] = clase_mayoritaria.drop(columns=str(data_file).split('_')[-1][:-4]).apply(


...

C:\Users\Ceiec09\AppData\Local\Temp\ipykernel_7884\1170576296.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clase_mayoritaria['distancia'] = clase_mayoritaria.drop(columns=str(data_file).split('_')[-1][:-4]).apply(
C:\Users\Ceiec09\AppData\Local\Temp\ipykernel_7884\1170576296.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clase_mayoritaria['distancia'] = clase_mayoritaria.drop(columns=str(data_file).split('_')[-1][:-4]).apply(


SMOTE y downsampling no aleatorio aplicados. Dataset guardado como: ./entrenamiento_balanceado\balanceado_living_data_with_Ideación paranoide.csv
SMOTE y downsampling no aleatorio aplicados. Dataset guardado como: ./entrenamiento_balanceado\balanceado_living_data_with_Obsesión-compulsión.csv


C:\Users\Ceiec09\AppData\Local\Temp\ipykernel_7884\1170576296.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clase_mayoritaria['distancia'] = clase_mayoritaria.drop(columns=str(data_file).split('_')[-1][:-4]).apply(
C:\Users\Ceiec09\AppData\Local\Temp\ipykernel_7884\1170576296.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clase_mayoritaria['distancia'] = clase_mayoritaria.drop(columns=str(data_file).split('_')[-1][:-4]).apply(


SMOTE y downsampling no aleatorio aplicados. Dataset guardado como: ./entrenamiento_balanceado\balanceado_living_data_with_Psicoticismo.csv
SMOTE y downsampling no aleatorio aplicados. Dataset guardado como: ./entrenamiento_balanceado\balanceado_living_data_with_Sensibilidad interpersonal.csv
SMOTE y downsampling no aleatorio aplicados. Dataset guardado como: ./entrenamiento_balanceado\balanceado_living_data_with_Somatización.csv


C:\Users\Ceiec09\AppData\Local\Temp\ipykernel_7884\1170576296.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clase_mayoritaria['distancia'] = clase_mayoritaria.drop(columns=str(data_file).split('_')[-1][:-4]).apply(


# 1. Modelos de clasificación Binaria

Decision Trees

In [13]:
# Diccionarios de hiperparámetros para cada modelo
param_grids = {
    'DecisionTree': {
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'criterion': ['gini', 'entropy']
    },
    'RandomForest': {
        'n_estimators': [50, 100, 150],
        'max_depth': [None, 10, 20],
        'min_samples_split': [2, 5],
        'min_samples_leaf': [1, 2]
    },
    'SVM': {
        'C': [0.1, 1, 10],
        'kernel': ['linear', 'rbf', 'poly'],
        'degree': [3, 4, 5],
        'gamma': ['scale', 'auto']
    },
    'GradientBoosting': {
        'n_estimators': [50, 100, 150],
        'learning_rate': [0.01, 0.1, 0.2],
        'max_depth': [3, 4, 5],
        'subsample': [0.8, 1.0]
    },
    'NaiveBayes': {
        'var_smoothing': np.logspace(0, -9, num=10),
    },
    'LogisticRegression': [
        {
            'C': [0.01, 0.1, 1, 10, 100],
            'penalty': ['l2'],
            'solver': ['lbfgs'],
            'max_iter': [100, 200, 500]
        },
        {
            'C': [0.01, 0.1, 1, 10, 100],
            'penalty': ['l1'],
            'solver': ['liblinear'],
            'max_iter': [100, 200, 500]
        },
        {
            'C': [0.01, 0.1, 1, 10, 100],
            'penalty': ['l2'],
            'solver': ['liblinear'],
            'max_iter': [100, 200, 500]
        }
    ],
    'MLP': {
        'hidden_layer_sizes': [(50,), (100,), (50, 50)],
        'activation': ['relu', 'tanh'],
        'solver': ['adam', 'sgd'],
        'alpha': [0.0001, 0.001],
        'learning_rate': ['constant', 'adaptive']
    }
}

In [14]:
# Clasificadores
classifiers = {
    'DecisionTree': DecisionTreeClassifier(random_state=42),
    'RandomForest': RandomForestClassifier(random_state=42),
    'SVM': SVC(probability=True, random_state=42),  # Se necesita probability=True para SHAP
    'GradientBoosting': GradientBoostingClassifier(random_state=42),
    'NaiveBayes': GaussianNB(),
    'LogisticRegression': LogisticRegression(random_state=42, max_iter=1000),
    'MLP': MLPClassifier(random_state=42, max_iter=1000)
}

In [15]:
# Función para calcular sensibilidad y especificidad
def calcular_sensibilidad_especificidad_conjunto(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    sensibilidad = tp / (tp + fn) if (tp + fn) > 0 else 0
    especificidad = tn / (tn + fp) if (tn + fp) > 0 else 0
    return sensibilidad, especificidad

In [16]:
# Función para calcular los valores SHAP
def calcular_shap_values(clasificador, X_train, X_test, nombre_algoritmo):
    explainer = None

    # Reducir el fondo para la explicación
    background = shap.sample(X_train, 33)  # Reducir si el conjunto es grande

    # Seleccionar el tipo de explicador según el modelo
    if nombre_algoritmo in ['SVM', 'NaiveBayes', 'LogisticRegression', 'MLP']:
        explainer = shap.KernelExplainer(clasificador.predict_proba, background)
    elif nombre_algoritmo in ['RandomForest', 'DecisionTree', 'GradientBoosting']:
        explainer = shap.TreeExplainer(clasificador)

    # Calcular valores SHAP
    shap_values = explainer.shap_values(X_test)

    # Verificar si shap_values es una lista (caso de clasificación binaria)
    if isinstance(shap_values, list):
        # Seleccionar los valores SHAP de la clase positiva (índice 1)
        shap_values = shap_values[1]
    
    # Verificar dimensiones del resultado y ajustar
    if shap_values.ndim == 3:  # Caso raro con dimensiones adicionales
        shap_values = shap_values[:, :, 1]  # Seleccionar la clase positiva

    # Crear el DataFrame SHAP
    shap_df = pd.DataFrame(shap_values, columns=X_test.columns)

    return shap_df

In [17]:
# Directorio de datos
data_dir = './entrenamiento_balanceado'

datasets = os.listdir(data_dir)

datasets = [x for x in datasets if x.endswith('.csv')]

In [18]:
# Número de particiones (folds) en la validación cruzada
kf = KFold(n_splits=5, shuffle=True, random_state=42)

resultados_totales = []

In [19]:
resultados_totales = []
tiempos_clasificadores = {}

# Bucle principal
for dataset in tqdm(datasets, desc='Analizando', dynamic_ncols=True):
    # Cargar datos de entrenamiento
    datos_entrenamiento = pd.read_csv(f'./entrenamiento_balanceado/{dataset}', sep=';')
    
    # Extraer información de nombre del archivo
    subset = dataset.split('_')[1]
    issue = dataset.split('_')[-1].split('.')[0]
    column = str(dataset)[:-4].split('_')[-1]
    
    # Separar en características y etiquetas
    datos_entrenamiento_balanced = datos_entrenamiento.drop(columns=[column])
    labels_columna_balanced = datos_entrenamiento[column]

    # Dividir en conjunto de entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(
        datos_entrenamiento_balanced, labels_columna_balanced, test_size=0.2, random_state=42, stratify=labels_columna_balanced
    )

    train_size = len(X_train)
    test_size = len(X_test)
    train_class_distribution = y_train.value_counts(normalize=True).to_dict()
    test_class_distribution = y_test.value_counts(normalize=True).to_dict()
    
    # Evaluar cada clasificador
    for nombre_algoritmo, clasificador in classifiers.items():

        tiempo_inicio = datetime.datetime.now()
        
        # GridSearchCV para modelos con hiperparámetros
        if param_grids[nombre_algoritmo]:  
            grid_search = GridSearchCV(
                estimator=clasificador,
                param_grid=param_grids[nombre_algoritmo],
                scoring='accuracy',
                cv=5,
                n_jobs=-1,
                return_train_score=True,
                error_score='raise'
            )

            grid_search.fit(X_train, y_train)
            mejor_modelo = grid_search.best_estimator_
            mejores_params = grid_search.best_params_

            mean_train_accuracy = np.mean(grid_search.cv_results_['mean_train_score'])
            std_train_accuracy = np.std(grid_search.cv_results_['mean_train_score'])
            mean_val_accuracy = np.mean(grid_search.cv_results_['mean_test_score'])
            std_val_accuracy = np.std(grid_search.cv_results_['mean_test_score'])
            
        else:  
            mejor_modelo = clasificador.fit(X_train, y_train)
            mejores_params = "N/A"
            mean_train_accuracy = accuracy_score(y_train, mejor_modelo.predict(X_train))
            std_train_accuracy = 0
            mean_val_accuracy = accuracy_score(y_test, mejor_modelo.predict(X_test))
            std_val_accuracy = 0
        
        # Evaluación final en conjunto de prueba
        predicciones_test = mejor_modelo.predict(X_test)
        accuracy_test = accuracy_score(y_test, predicciones_test)
        sensibilidad_test, especificidad_test = calcular_sensibilidad_especificidad_conjunto(y_test, predicciones_test)

        # Cálculo de los valores SHAP
        shap_df = calcular_shap_values(mejor_modelo, X_train, X_test, nombre_algoritmo)

        # Medir uso de RAM del modelo
        modelo_size_ram = asizeof.asizeof(mejor_modelo)

        # Guardar modelo en disco y medir tamaño
        model_path = f'model_{nombre_algoritmo}_{dataset}.joblib'
        joblib.dump(mejor_modelo, model_path)
        model_size_disk = os.path.getsize(model_path)

        # Calcular Sample Size
        sample_size_train = len(X_train)  # Número de muestras usadas para entrenamiento
        sample_size_test = len(X_test)   # Número de muestras usadas para prueba
       
        
        validation_folds = grid_search.cv
        val_size_per_fold = train_size // validation_folds
        
        # Agregar cada fila al conjunto de resultados
        for idx, row in shap_df.iterrows():
            resultados_totales.append({
                'dataset': dataset,
                'subset': subset,
                'train_size': train_size,
                'test_size': test_size,
                'validation_folds': validation_folds,
                'val_size_per_fold': val_size_per_fold,
                'train_class_distribution': train_class_distribution,
                'test_class_distribution': test_class_distribution,
                'issue': issue,
                'algorithm': nombre_algoritmo,
                'best_params': mejores_params,
                'mean_train_accuracy': mean_train_accuracy,
                'std_train_accuracy': std_train_accuracy,
                'mean_val_accuracy': mean_val_accuracy,
                'std_val_accuracy': std_val_accuracy,
                'accuracy_test': accuracy_test,
                'sensitivity_test': sensibilidad_test,
                'specificity_test': especificidad_test,
                'shap_feature_values': row.to_dict(),
                'sample_size_train': sample_size_train,
                'sample_size_test': sample_size_test
            })
        
        tiempo_fin = datetime.datetime.now()
        diferencia = tiempo_fin - tiempo_inicio

        # Registrar tiempos y tamaños en tiempos_clasificadores
        tiempos_clasificadores[f'{nombre_algoritmo}_{dataset}'] = {
            'Tiempo inicio': tiempo_inicio,
            'Tiempo fin': tiempo_fin,
            'Diferencia': diferencia,
            'Modelo RAM (MB)': modelo_size_ram / (1024 ** 2),
            'Modelo Disco (MB)': model_size_disk / (1024 ** 2),
            'Sample Size Entrenamiento': sample_size_train,
            'Sample Size Prueba': sample_size_test
        }

Analizando:   0%|          | 0/36 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

...

  0%|          | 0/64 [00:00<?, ?it/s]

  0%|          | 0/64 [00:00<?, ?it/s]

  0%|          | 0/64 [00:00<?, ?it/s]

  0%|          | 0/64 [00:00<?, ?it/s]

Analizando: 100%|██████████| 36/36 [1:04:49<00:00, 108.04s/it]


In [1]:
display(tiempos_clasificadores)

NameError: name 'tiempos_clasificadores' is not defined

In [21]:
# Guardar resultados en un único CSV
resultados_df = pd.DataFrame(resultados_totales)
resultados_df.to_csv('resultados_completos.csv', index=False)

In [27]:
# Cargar el archivo CSV de resultados
archivo = 'resultados_completos.csv'
df_resultados = pd.read_csv(archivo)

# Agrupar por `etiqueta` y `subset`, y seleccionar el modelo con mayor `Accuracy Test` en cada grupo
mejores_modelos = df_resultados.loc[df_resultados.groupby(['issue', 'subset'])['accuracy_test'].idxmax()]

# Mostrar los mejores modelos para cada combinación de `etiqueta` y `subset`
print(mejores_modelos)

# Guardar los resultados en un nuevo archivo CSV si deseas conservarlos
mejores_modelos.to_csv("mejores_modelos_por_etiqueta_subset.csv", index=False, sep=';')


                                                 dataset       subset  \
50       balanceado_demographic_data_with_Agorafobia.csv  demographic   
3936        balanceado_economic_data_with_Agorafobia.csv     economic   
7872          balanceado_health_data_with_Agorafobia.csv       health   
11808         balanceado_living_data_with_Agorafobia.csv       living   
486        balanceado_demographic_data_with_Ansiedad.csv  demographic   
4254          balanceado_economic_data_with_Ansiedad.csv     economic   
8090            balanceado_health_data_with_Ansiedad.csv       health   
11994           balanceado_living_data_with_Ansiedad.csv       living   
954       balanceado_demographic_data_with_Depresión.csv  demographic   
4662         balanceado_economic_data_with_Depresión.csv     economic   
8690           balanceado_health_data_with_Depresión.csv       health   
12718          balanceado_living_data_with_Depresión.csv       living   
1274     balanceado_demographic_data_with_Hostilida

In [23]:
hora_fin_codigo = datetime.datetime.now()
print(hora_fin_codigo)
print(hora_fin_codigo - hora_inicio_codigo)

2024-11-22 11:05:02.313203
1:05:03.152139
